# Database Builder Example - Charleston, USA

In this notebook we are going to create a complete **database** for our FloodAdapt model in Charleston, USA. 
To do so we use the in FloodAdapt integrated **Database builder**, which allows for a quick and easy database creation!
We presume that you have already created a functioning [**Delft-FIAT**](https://github.com/Deltares/Delft-FIAT) and [**SFINCS model**](https://github.com/Deltares/SFINCS) and we'll go from there. 


## **Step 0**: Import packages

In [ ]:
# Import packages
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from shapely.geometry import Polygon
import toml

import flood_adapt.database_builder.database_builder as db
import flood_adapt.adapter.fiat_adapter as fiat
from flood_adapt import unit_system as us

%matplotlib inline

## **Step 1**: Explore the Delft-FIAT model

The study area is in **Charleston, USA**, a coastal city on the East Coast of the United States. 
So, let's first inspect the exposure data in the **Delft-FIAT model** to get an understanding of our study area.  

We initiate the **FiatAdapter** in FloodAdapt with the model root of the Delft-FIAT model and explore the exposure data in the map.

In [ ]:
# Initiate the FiatAdapter class from FloodAdapt and plot the Fiat model. 
root = Path(r".\data\nb1_database_builder") # The root folder of this example
fn_fiat = Path(root / "fiat") 
fa = fiat.FiatAdapter(
    model_root =  fn_fiat
    )

# Get the geodataframe with exposure data
gdf = fa._model.exposure.get_full_gdf(fa._model.exposure.exposure_db)

# Plot the region and the secondary_object_types of the exposure data
m = gdf.explore(column="secondary_object_type", name="Exposure types")
m

## **Step 2**: Database Builder Configuration

The database builder needs simply a **configuration file** as **input** to create the database. The configuration file is a simple text file and can be created manually. In the next steps we will show you how you can create this configuration file. 
So, now that we got an understanding of our study area and our exposure data we are ready to build our base **configuration file** for the **DatabaseBuilder**!

The configuration file consists of:
1. **Basic model parameters** to create a simple FloodAdapt database.
2. **Advance model parameters** to create a more complex FloodAdapt database.

In this step we will look at the basic model parameters.  
- **name**: The name of the database to be created
- **database_path**: The path where to save the database (if not provided, a database folder will be created at the location of this condig file)
- **sfincs**: The path to the overland SFINCS model
- **fiat**: The  path to the FIAT model
- **unit_system**: metric or imperial  
* [gui] The values below define the GUI variables. They are used as the maximum values to scale the mapbox layers. Same units as in the models should be used. You can adjust the values, so that they are sensible with respect to your data.
- **max_flood_depth** = 5 
- **max_aggr_dmg** = 10000000
- **max_footprint_dmg** = 250000
- **max_benefits** = 50000000

You can either create the configuration file manually with a dictionary or use the integrated classes in FloodAdapt for a more streamlined approach. 
We show you both options below.

### **Step 2a**: Create a configuration file from a dictionary

In [ ]:
# Create the configuration file for the database builder for a simple FloodAdapt database. 

# all paths should be provided with forward slashes (/)
config = {
"name": "charleston_example1",
"database_path": "Database",
"sfincs_overland" : { 
    "name":"../../../data/nb1_database_builder/sfincs",
    "reference":"MSL"
    },
"fiat" : "../../../data/nb1_database_builder/fiat",
"unit_system" :"imperial",
"gui": {
"max_flood_depth": 5,
"max_aggr_dmg" : 10000000,
"max_footprint_dmg": 250000,
"max_benefits" : 50000000}
}

# Save the configuration file
with open("./data/nb1_database_builder/output/db_config.toml", "w") as f:
    toml.dump(config, f)

### **Step 2b**: Create a configuration file from the Database Builder classes

**Important!**  
The water level reference should be set to the reference of your DEM. You can create a reference system manually as shown below or fetch this information from a close-by tide gauge. This will be shown in the more advanced options. 


In [ ]:
config_model = db.ConfigModel(
    name="charleston_example2",
    database_path= str((root /"output"/"Database").absolute()),
    unit_system= db.UnitSystems.imperial,
    gui=db.GuiConfigModel(
        max_flood_depth=5,
        max_aggr_dmg=1e6,
        max_footprint_dmg=250000,
        max_benefits=5e6,
    ),
    sfincs_overland=db.FloodModel(
        name=str((root / "sfincs").absolute()),
        reference="MSL",
    ),
    references=db.WaterlevelReferenceModel(
        reference="MSL",
        datums=[
            db.DatumModel(name="MSL", height=us.UnitfulLength(value=0, units=us.UnitTypesLength.meters)),
            db.DatumModel(name="NAVD88", height=us.UnitfulLength(value=1, units=us.UnitTypesLength.meters))
        ]
    ),
    fiat=str((root / "fiat").absolute()),
)

In [ ]:
config_model

## **Step 3**: Run the Database Builder

We are ready to run the Database Builder with the configuration file we just created above.

In [ ]:
config_path = root / "output" / "db_config.toml"
config = db.ConfigModel.read(config_path)
dbs = db.DatabaseBuilder(config)
dbs.build(overwrite = True)

In [ ]:
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)